# Week 12: Generative Agents Demo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Digital-AI-Finance/agentic-artificial-intelligence/blob/main/L12_Research_Frontiers/L12_Generative_Agents.ipynb)

Simplified demonstration of generative agent concepts from Park et al. (2023).

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install -q langchain-openai python-dotenv
    from google.colab import userdata
    import os
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
from typing import List, Dict
from dataclasses import dataclass, field
from datetime import datetime
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
print("Ready")

## 1. Memory Stream

In [ ]:
@dataclass
class Memory:
    content: str
    timestamp: str
    importance: float = 5.0
    
class MemoryStream:
    def __init__(self):
        self.memories: List[Memory] = []
    
    def add(self, content: str, importance: float = 5.0):
        self.memories.append(Memory(
            content=content,
            timestamp=datetime.now().strftime("%H:%M"),
            importance=importance
        ))
    
    def retrieve(self, query: str, k: int = 5) -> List[Memory]:
        # Simple recency + importance retrieval
        scored = [(m, m.importance + (len(self.memories) - i) * 0.1) 
                  for i, m in enumerate(self.memories)]
        scored.sort(key=lambda x: x[1], reverse=True)
        return [m for m, _ in scored[:k]]
    
    def __len__(self):
        return len(self.memories)

## 2. Generative Agent

In [ ]:
@dataclass
class GenerativeAgent:
    name: str
    description: str
    memory: MemoryStream = field(default_factory=MemoryStream)
    
    def perceive(self, observation: str):
        """Add observation to memory with importance scoring."""
        importance_prompt = f"""Rate importance of this observation for {self.name} (1-10):
Agent description: {self.description}
Observation: {observation}
Return just the number."""
        
        try:
            importance = float(llm.invoke(importance_prompt).content.strip())
        except:
            importance = 5.0
        
        self.memory.add(observation, importance)
        print(f"[{self.name}] Observed: {observation[:50]}... (importance: {importance:.1f})")
    
    def reflect(self) -> str:
        """Generate high-level reflections from memories."""
        recent_memories = self.memory.retrieve("", k=10)
        memory_text = "\n".join([m.content for m in recent_memories])
        
        prompt = f"""Based on these recent memories, generate a high-level insight for {self.name}.

Agent: {self.description}
Recent memories:
{memory_text}

What is a key insight or reflection?"""
        
        reflection = llm.invoke(prompt).content
        self.memory.add(f"Reflection: {reflection}", importance=8.0)
        return reflection
    
    def plan(self, goal: str) -> List[str]:
        """Generate a plan for achieving a goal."""
        relevant = self.memory.retrieve(goal, k=5)
        context = "\n".join([m.content for m in relevant])
        
        prompt = f"""Create a simple plan for {self.name} to achieve this goal.

Agent: {self.description}
Goal: {goal}
Relevant context: {context}

List 3-5 steps:"""
        
        response = llm.invoke(prompt).content
        steps = [s.strip() for s in response.split('\n') if s.strip()]
        return steps[:5]
    
    def react(self, situation: str) -> str:
        """React to a situation based on personality and memories."""
        relevant = self.memory.retrieve(situation, k=3)
        context = "\n".join([m.content for m in relevant])
        
        prompt = f"""How would {self.name} react to this situation?

Agent: {self.description}
Relevant memories: {context}
Situation: {situation}

Describe their reaction briefly:"""
        
        return llm.invoke(prompt).content

## 3. Simulate Agent Behavior

In [ ]:
# Create an agent
alice = GenerativeAgent(
    name="Alice",
    description="A curious researcher interested in AI and machine learning. She is thoughtful and analytical."
)

# Add some initial memories
alice.perceive("Started working on a new research project about language models")
alice.perceive("Had a great discussion with Bob about agent architectures")
alice.perceive("Read an interesting paper about generative agents")
alice.perceive("Noticed that multi-agent systems might be the future")

print(f"\nAlice has {len(alice.memory)} memories")

In [ ]:
# Generate reflection
reflection = alice.reflect()
print(f"\nAlice's reflection:\n{reflection}")

In [ ]:
# Generate a plan
plan = alice.plan("Write a paper about agent architectures")
print("\nAlice's plan:")
for i, step in enumerate(plan, 1):
    print(f"  {i}. {step}")

In [ ]:
# React to a situation
reaction = alice.react("A colleague asks for help debugging their agent code")
print(f"\nAlice's reaction:\n{reaction}")

## Summary

Demonstrated core generative agent concepts:
- **Memory Stream**: Timestamped observations with importance
- **Reflection**: High-level insights from memories
- **Planning**: Goal-directed behavior generation
- **Reaction**: Context-aware responses

Full implementation would add: spatial awareness, social interactions, and emergent behavior.